In [ ]:
import torch
import transformers
import pandas as pd
import numpy as np
from argparse import Namespace
from tqdm.auto import tqdm
from datasets import load_from_disk
from transformers import AutoTokenizer

In [ ]:
ds = load_from_disk("/data3/mmendieta/Violence_data/geo_corpus.0.0.1_dataset_for_train_all_labels")

In [ ]:
ds['train'][0]

In [ ]:
# Possible values
# Smaller-LABSE: setu4993/smaller-LaBSE
# LABSE: setu4993/LaBSE
# XLMT: cardiffnlp/twitter-xlm-roberta-base-sentiment

config = {
    "model_ckpt": "setu4993/LaBSE",
    "batch_size": 1024,
    "num_labels" : 40,
    "max_length": 32,
    "seed": 42,
    "fout": "/data3/mmendieta/Violence_data/geo_corpus.0.0.1_tok_ds_labse_all_labels"
}

args = Namespace(**config)

In [ ]:
# Instantiate the tokenizer
model_ckpt = args.model_ckpt
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, 
                                              model_max_length=args.max_length)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

In [ ]:
%time tokenized_ds = ds.map(tokenize, batched=True)

In [ ]:
tokenized_ds = tokenized_ds.remove_columns('text')

In [ ]:
tokenized_ds.set_format('torch')

In [ ]:
tokenized_ds

In [ ]:
tokenized_ds["train"].features

In [ ]:
tokenized_ds.save_to_disk(args.fout)